In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark
!pip install numpy
!pip install Pillow
!pip install petastorm
!pip install hyperopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=9df92f869bf78bfebbff08a9c1feb9ce338c606b5553e06ac8fb61517d504108
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.8 MB/s eta 0:00:00


In [3]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType

In [4]:
from petastorm.codecs import ScalarCodec, CompressedImageCodec, NdarrayCodec
from petastorm.etl.dataset_metadata import materialize_dataset
from petastorm.unischema import dict_to_spark_row, Unischema, UnischemaField

In [5]:
from petastorm.spark import SparkDatasetConverter, make_spark_converter
from petastorm import TransformSpec

In [6]:
from pyspark.sql.functions import col
from pyspark.sql.types import *

In [7]:
from pyspark.sql.types import (ArrayType, BinaryType, BooleanType, ByteType, DoubleType, FloatType,
                               IntegerType, LongType, ShortType, StringType, StructField, StructType)

In [8]:
# Start Spark session

spark = SparkSession.builder.appName("Create petastorm store").config("spark.memory.offHeap.enabled", True) \
                            .config("spark.memory.offHeap.size", "30g").getOrCreate()

spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, 'file:/content/drive')

## Understand Primitives

In [9]:
schema = StructType([
    StructField("bool_col", BooleanType(), False),
    StructField("float_col", FloatType(), False),
    StructField("double_col", DoubleType(), False),
    StructField("short_col", ShortType(), False),
    StructField("int_col", IntegerType(), False),
    StructField("long_col", LongType(), False),
    StructField("str_col", StringType(), False),
    StructField("bin_col", BinaryType(), False),
    StructField("byte_col", ByteType(), False),

])

In [13]:
df = spark.createDataFrame(
    [(True, 0.12, 432.1, 5, 5, 0, "hello", bytearray(b"spark\x01\x02"), -128),
    (False, 123.45, 0.987, 9, 908, 765, "petastorm", bytearray(b"\x0012345"), 127)],
     schema=schema).coalesce(1)

# if we use numPartition > 1 in coalesce, the order of the loaded dataset would be non-deteministic
# just for the learning phase - don't use in production
expected_df = df.collect()

In [14]:
expected_df

[Row(bool_col=True, float_col=0.11999999731779099, double_col=432.1, short_col=5, int_col=5, long_col=0, str_col='hello', bin_col=bytearray(b'spark\x01\x02'), byte_col=-128),
 Row(bool_col=False, float_col=123.44999694824219, double_col=0.987, short_col=9, int_col=908, long_col=765, str_col='petastorm', bin_col=bytearray(b'\x0012345'), byte_col=127)]

## Test Tensorflow dataset

In [15]:
import tensorflow as tf

In [ ]:
import tensorflow as tf


converter = make_spark_converter(df)
with converter.make_tf_dataset() as dataset:
        iterator = iter(dataset)
        print(iterator.element_spec)

        tensor = iterator.get_next()
        print(tensor)